# Data Pre-Processing

## Libraries

We start by importing the necessary libraries. Pandas and NumPy for data manipulation, and statsmodels for statistical tests.

In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from scipy.interpolate import CubicSpline

import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import MinMaxScaler

import sys
sys.path.append('../')

from src.modwt import *

import pywt
from pywt import wavedec
from pywt import Wavelet

In [2]:
ref_data_range = "1H"
from_date = '2021-12-31 00:00:00' # 2020-12-31 00:00:00
until_date = '2025-07-31 00:00:00'
until_date_2 = '2025-08-01 00:00:00'

In [3]:
# list_data_name = ["BTCUSDT_1h", "ETHUSDT_1h", "BNBUSDT_1h", "XRPUSDT_1h", "ADAUSDT_1h", "LTCUSDT_1h"]
list_data_name = ["ADAUSDT_1h"]

In [ ]:
for data_name in list_data_name:

    # Import data
    ccy = data_name.split('_')[0]

    # data = pd.read_csv(f'../data/{data_name}_data_from_20180101_to_20230102.csv', parse_dates=['date'], index_col='date')
    data = pd.read_csv(f'../data/{data_name}_data_from_20210101_to_20251031.csv', parse_dates=['date'], index_col='date')

    # Get data from_date until until_date inclusive
    data = data[(data.index >= from_date) & (data.index <= until_date_2)]

    # Add column with original close
    data['original_close'] = data['close']

    # Handling missing data 

    # Check that there is no missing data
    # data.index = pd.to_datetime(data.index)
    my_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq=ref_data_range)

    # Get all the dates in the start_date and end_date range
    df_full_dates = pd.DataFrame()
    df_full_dates['date'] = my_range

    # Remove the data['date'] values that are not in df_full_dates['date']
    data = data[data.index.isin(df_full_dates['date'])]

    data_new = data.join(df_full_dates.set_index('date'), on='date', how='outer')
    data_new.sort_values(by=['date'], inplace=True)

    # data_new['date'] = pd.to_datetime(data_new['date'])  # Convert 'date' column to datetime type if needed
    data = data_new.copy()  # Make a copy of the dataframe
    # data = data.set_index('date')  # Set 'date' as the index for convenient time-based operations
    # data_filled = data_new.fillna(method='ffill')
    # data_filled = data_new.reset_index()    

    my_range_new = pd.date_range(start=data.index.min(), end=data.index.max(), freq=ref_data_range)

    data_interp_temp = data.copy()
    data_interp_temp.reset_index(inplace=True, drop=True)

    data_interp_temp['date_ordinal'] = data_interp_temp['date'].apply(lambda x: x.timestamp())

    known_data = data_interp_temp.dropna()

    cs = CubicSpline(known_data['date_ordinal'], known_data['close'])

    missing_data_indices = data_interp_temp[data_interp_temp['close'].isnull()].index

    interpolated_values = cs(data_interp_temp.loc[missing_data_indices, 'date_ordinal'])

    data_interp_temp.loc[missing_data_indices, 'close'] = interpolated_values

    data_interp_temp.set_index('date', inplace=True)

    data = data_interp_temp.copy()
    # replace the original data with the interpolated data by using the index
    data['processed_close'] = data_interp_temp['close']

    # add log return
    data['processed_log_return'] = np.log(data['close'] / data['close'].shift(1))

    # Remove the NaN values
    data = data[~data['processed_log_return'].isnull()]

    # Outliers

    # Compute Z-score for the 'close' price column
    z_scores = np.abs((data['processed_log_return'] - data['processed_log_return'].mean()) / data['processed_log_return'].std())

    # Assuming df is your DataFrame and 'log_return' is the column with outliers
    upper_bound = data['processed_log_return'].quantile(0.98)
    lower_bound = data['processed_log_return'].quantile(0.02)

    # Applying capping and flooring
    data['outliers_processed_log_return'] = data['processed_log_return'].clip(lower=lower_bound, upper=upper_bound)

    # z_scores = np.abs((data['outliers_processed_log_return'] - data['outliers_processed_log_return'].mean()) / data['outliers_processed_log_return'].std())

    # Normalization

    # Create a MinMaxScaler object
    scaler = MinMaxScaler(feature_range=(-1, 1))
    data[['normalized_outliers_processed_log_return']] = scaler.fit_transform(data[['outliers_processed_log_return']])

    # Denoising

    wt = modwt(data['normalized_outliers_processed_log_return'], 'db2', 5)
    wtmra = modwtmra(wt, 'db2')

    df_wtmra = pd.DataFrame(wtmra)
    df_wtmra = df_wtmra.transpose()

    # Rename the df_wtrma columns
    df_wtmra.columns = ['processed_log_return_wtmra_0', 'processed_log_return_wtmra_1', 'processed_log_return_wtmra_2', 'processed_log_return_wtmra_3', 'processed_log_return_wtmra_4', 'processed_log_return_wtmra_5']

    df_wtmra['processed_log_return_wtmra_5_4'] = df_wtmra['processed_log_return_wtmra_5'] + df_wtmra['processed_log_return_wtmra_4']
    df_wtmra['processed_log_return_wtmra_5_4_3'] = df_wtmra['processed_log_return_wtmra_5_4'] + df_wtmra['processed_log_return_wtmra_3']
    df_wtmra['processed_log_return_wtmra_5_4_3_2'] = df_wtmra['processed_log_return_wtmra_5_4_3'] + df_wtmra['processed_log_return_wtmra_2']
    df_wtmra['processed_log_return_wtmra_5_4_3_2_1'] = df_wtmra['processed_log_return_wtmra_5_4_3_2'] + df_wtmra['processed_log_return_wtmra_1']
    df_wtmra['processed_log_return_wtmra_5_4_3_2_1_0'] = df_wtmra['processed_log_return_wtmra_5_4_3_2_1'] + df_wtmra['processed_log_return_wtmra_0']

    df_wtmra['processed_log_return_wtmra_0_1'] = df_wtmra['processed_log_return_wtmra_0'] + df_wtmra['processed_log_return_wtmra_1']
    df_wtmra['processed_log_return_wtmra_0_1_2'] = df_wtmra['processed_log_return_wtmra_0_1'] + df_wtmra['processed_log_return_wtmra_2']
    df_wtmra['processed_log_return_wtmra_0_1_2_3'] = df_wtmra['processed_log_return_wtmra_0_1_2'] + df_wtmra['processed_log_return_wtmra_3']
    df_wtmra['processed_log_return_wtmra_0_1_2_3_4'] = df_wtmra['processed_log_return_wtmra_0_1_2_3'] + df_wtmra['processed_log_return_wtmra_4']
    df_wtmra['processed_log_return_wtmra_0_1_2_3_4_5'] = df_wtmra['processed_log_return_wtmra_0_1_2_3_4'] + df_wtmra['processed_log_return_wtmra_5']

    # Add the data['date'] column to the df_wtmra DataFrame
    df_wtmra['date'] = data.index
    # Set the index of df_wtmra to be the date column
    df_wtmra.set_index('date', inplace=True)

    # Add df_wtmra colums to the data DataFrame by joining on the date index
    data = data.join(df_wtmra)

    # Remove NANs
    data.processed_close.dropna(inplace=True)

    # Get data from_date until until_date inclusive
    data = data[(data.index >= from_date) & (data.index <= until_date)]


    # Save the processed data
    data.to_csv(f'../data/01-output-{data_name}-from-{from_date}-until-{until_date}-log-return.csv', index=True)

/var/folders/s0/_j22qz4n5j72grfwwy9xgbxc0000gn/T/ipykernel_97728/3778662857.py:19: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  my_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq=ref_data_range)
/var/folders/s0/_j22qz4n5j72grfwwy9xgbxc0000gn/T/ipykernel_97728/3778662857.py:37: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  my_range_new = pd.date_range(start=data.index.min(), end=data.index.max(), freq=ref_data_range)
